Wczytywanie zbioru danych.

In [23]:
import numpy as np
import pandas as pd

df = pd.read_csv("reviews_bow.csv")
# df = pd.read_csv("reviews_w2v.csv")
df.head()

,helpfulP,asin,helpfulN,unixReviewTime,reviewerID,score,reviewerName,0,1,2,...,290,291,292,293,294,295,296,297,298,299
0,6,2747,7,1333843200,30124,1.0,43149,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,0,5394,6,1388188800,4713,1.0,31532,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,2,7167,4,1356048000,59783,1.0,46230,0,0,0,...,0,0,0,0,0,0,0,0,2,0
3,2,4036,4,1334188800,67587,1.0,41970,0,0,0,...,0,0,0,0,0,0,0,0,2,0
4,0,4625,1,1387670400,65465,1.0,54619,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Wybór klasyfikatora i poszukiwanie optymalnych parametrów.

In [24]:
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import f1_score

y = df["score"]
X = df[list(set(df.columns) - {"score"})]
# print("X columns:", list(X.columns), "\ny columns:", y.name)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

print("\n---Tuning hyperparameters---")

param_grid = {
    'n_estimators': [200, 350, 500],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth': [5, 6, 7, 8, 9, 10],
    'n_jobs': [-1],
    'random_state': [23]
}

gs = GridSearchCV(RandomForestClassifier(), param_grid, n_jobs=-1, verbose=1, scoring='f1_weighted')
gs.fit(X_train, y_train)
print(gs.best_params_)


---Tuning hyperparameters---
Fitting 5 folds for each of 54 candidates, totalling 270 fits


KeyboardInterrupt: 

Rezultaty klasyfikacji i porównania z algorytmem większościowym oraz losowym.

In [ ]:
print("\n---Training classifier with best found parameters---")
clf = RandomForestClassifier(**gs.best_params_)
clf.fit(X_train, y_train)

Ocena klasyfikatora oraz porównanie go do algorytmów losowego oraz większościowego.

In [ ]:
import random
from sklearn.metrics import f1_score, precision_score, recall_score


def scores(y_true, y_pred, name=None, df_compare=pd.DataFrame(np.zeros((5, 3)))):
    measures = ["F1", "Precision", "Recall"]
    df_scores = pd.DataFrame(columns=measures, index=list_of_labels)

    df_scores["F1"] = f1_score(y_true, y_pred, average=None, pos_label=None, labels=list_of_labels)
    df_scores["Precision"] = precision_score(y_true, y_pred, average=None, pos_label=None, labels=list_of_labels,
                                             zero_division=0)
    df_scores["Recall"] = recall_score(y_true, y_pred, average=None, pos_label=None, labels=list_of_labels)

    print(f"=================== Results: {name} ===================")
    # print(pd.DataFrame(df_scores.to_numpy() - df_compare.to_numpy()).T)
    print(df_scores.T)
    # return df_scores


list_of_labels = y_train.unique()
majority_label = y_train.value_counts().index[0]
random.seed(23)

y_pred_random, y_pred_majority = [], []

for _ in range(len(X_test)):
    y_pred_random.append(random.choice(list_of_labels))
    y_pred_majority.append(majority_label)

scores(y_test, y_pred_random, name="RandomClassifier")
scores(y_test, y_pred_majority, name="MajorityClassifier")
scores(y_test, clf.predict(X_test), name="Trained Classifier")
print("=================== Results are difference to random algorithm ===================")

Uczenie klasyfikatora z najlepszymi hiperparametrami na pełnym zbiorze oraz jego zapis do pliku.

In [ ]:
import pickle

clf.fit(X, y)

with  open("clf.pkl", "ab") as f:
    pickle.dump(clf, f)